# Introduction: Band-Pass Filtering for Lung Sound Analysis

This notebook demonstrates how to apply a Butterworth band-pass filter to lung sound recordings using Python.
Band-pass filtering is a common preprocessing step in biomedical signal analysis, used to remove very low and very high frequency noise and keep only the relevant frequency range of interest.

## Filter Parameters

* `fc = (low_cut, high_cut):`
Defines the frequency range (in Hz) that passes through the filter.
For example, `fc = (50, 100)` keeps frequencies between 50 Hz and 100 Hz. Which would be used for heartsounds, but not for lungsounds. The range for that would be between 80 - 3000Hz for example.

* `order:`
Determines how steep the filter’s roll-off is. A higher order gives stronger attenuation outside the passband.
Here we use `order = 2`.

* `zero_phase:`
If `True`, the filter is applied forward and backward using `sosfiltfilt`, which cancels any phase distortion (zero group delay).
This makes the filtered signal better aligned in time with the original waveform — important for lung sound analysis.

## Usage Overview

1. Load a lung sound signal x (e.g. from a .wav file).
2. Apply the filter:
    ```python
    y = bandpass_filter(x, Fs, fc=(50, 100), order=2, zero_phase=True)
    ```
3. Optionally, visualize x and y to see the filtering effect, or compute spectral differences.

This example serves as a compact demonstration of how filtering can improve signal clarity.

In [1]:
import numpy as np
from scipy.signal import butter, sosfiltfilt, sosfilt

def bandpass_filter(x, fs, fc=(50.0, 100.0), order=2, zero_phase=True, axis=-1):
    """
    Apply a Butterworth band-pass filter to a 1D or multi-dimensional signal.

    A band-pass filter allows only frequencies between `low_cut` and `high_cut`
    to pass through and removes the rest. This is useful for isolating
    specific frequency components — for example, parts of lung or heart sounds.

    Parameters
    ----------
    x : array_like
        Input signal (can be 1D or multi-dimensional).
        Filtering is applied along the specified `axis`.
    fs : float
        Sampling frequency of the signal in Hz.
    fc : (float, float)
        Tuple containing (low_cut, high_cut) in Hz.
        Defines the frequency band to keep.
    order : int
        Filter order per section.
        A higher order means a steeper cutoff between pass and stop bands.
        Note: When `zero_phase=True`, the filter is applied twice,
        effectively doubling the order in the frequency response.
    zero_phase : bool
        If True, applies the filter forward and backward (`sosfiltfilt`),
        which cancels any phase shift (zero delay in time).
        If False, applies the filter once (`sosfilt`), which can introduce delay.
    axis : int
        The axis along which to apply the filter (default is -1, the last axis).

    Returns
    -------
    y : ndarray
        Filtered signal. Has the same shape as the input `x`.
    """

    # Unpack the low and high cutoff frequencies
    low, high = fc

    # Safety check: cutoffs must be positive, and below the Nyquist frequency (fs/2)
    if not (0 < low < high < fs/2):
        raise ValueError(f"Cutoffs must satisfy 0 < {low=} < {high=} < Nyquist ({fs/2}).")

    # Normalize cutoff frequencies to the Nyquist frequency (fs/2)
    wn = (low / (fs / 2), high / (fs / 2))

    # Design the Butterworth band-pass filter (using second-order sections form)
    sos = butter(order, wn, btype='band', output='sos')

    # Apply the filter to the signal
    if zero_phase:
        # Zero-phase filtering (applied forward and backward)
        # No phase shift occurs, but filtering is effectively done twice
        return sosfiltfilt(sos, np.asarray(x), axis=axis)
    else:
        # Single-pass filtering (may introduce a time delay)
        return sosfilt(sos, np.asarray(x), axis=axis)


In [ ]:
# Example usage

# Sampling rate (samples per second)
Fs = 15750

# Band-pass frequency range (keep components between 50 Hz and 3000 Hz)
fc = (50, 3000)

# Suppose `x` is your raw audio data (as a NumPy array)
# For example, x could be loaded from a .wav file or sensor recording
# x = np.array([...])  # your raw data here

# Apply the band-pass filter
y = bandpass_filter(x, Fs, fc=fc, order=2, zero_phase=True)

# (Optional) Create a time axis in milliseconds for plotting or analysis
# The time axis matches the length of the signal
t_ms = np.arange(x.shape[-1]) * (1000.0 / Fs)